Librerias 

In [2]:
import numpy as np

Metrica Estimate of Positive Correctness

In [3]:
def epc_c(groups, classes):
    tot_samp = sum([len(group) for group in groups])
    epc = 0.0
    for group in groups:
        if len(group) == 0:
            continue
        score = 0.0
        for class_val in classes:
            proportion = [row[-1] for row in group].count(class_val) /len(group)
            score += proportion ** 2
        epc += score * (len(group)/ tot_samp)
    return epc


Variance reduction

In [4]:
def vari_red(groups):
    total_samples = sum([len(group) for group in groups])
    if total_samples == 0:
        return 0  
    total_variance = 0.0
    for group in groups:
        if len(group) == 0:  #
            continue
        variance = np.var([row[-1] for row in group]) 
        total_variance += len(group) * variance  
    return total_variance / total_samples

CART

In [5]:
def split_node(index, value, data):
    left, right = [], []
    for row in data:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right

def best_split(data, metric, is_clasification = True):
    class_values = list(set(row[-1] for row in data))
    b_index, b_value, b_score, b_groups = None, None, float('-inf') if is_clasification else float('inf'), None
    for index in range(len(data[0]) - 1):
        for row in data:
            groups = split_node(index, row[index], data)
            score = metric(groups, class_values) if is_clasification else vari_red(groups)
            if (is_clasification and score > b_score) or (not is_clasification and score < b_score):
                b_index, b_value, best_score, b_groups = index, row[index], score, groups
    return {'index': b_index, 'value': b_value, 'groups': b_groups}

def leaf(group):
    out = [row[-1] for row in group]
    return max(set(out), key=out.count)

def split(node, max_depth, min_size, depth, metric, is_clasification = True):
    left, right = node['groups']
    del(node['groups'])
    if not left or not right:
        node['left'] = node['right'] = leaf(left + right)
        return
    if depth >= max_depth:
        node['left'], node['right'] = leaf(left), leaf(right)
        return
    if len(left) <= min_size:
        node['left'] = leaf(left)
    else:
        node['left'] = best_split(left, metric, is_clasification)
        split(node['left'], max_depth, min_size, depth+1, metric, is_clasification)
    if len(right) <= min_size:
        node['right'] = leaf(right)
    else:
        node['right'] = best_split(right, metric, is_clasification)
        split(node['right'], max_depth, min_size, depth+1, metric, is_clasification) 

def tree(train, max_depth, min_size, metric, is_clasification=True):
    root = best_split(train, metric, is_clasification)
    split(root, max_depth, min_size, 1, metric, is_clasification)
    return root

def predction(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predction(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predction(node['right'], row)
        else:
            return node['right']

Ejemplo

In [6]:
data = [
    [2.5, 3.5, 1.5, 0],  
    [1.0, 2.0, 1.0, 0],
    [3.0, 4.0, 2.0, 0],
    [4.5, 5.5, 2.5, 1],
    [5.0, 6.0, 3.0, 1],
    [6.5, 7.5, 3.5, 1],
    [1.5, 2.5, 1.2, 0],
    [7.0, 8.0, 4.0, 1],
    [8.0, 9.0, 5.0, 1],
    [4.0, 2.0, 1.8, 0],
    [9.0, 7.5, 6.0, 1],
    [5.5, 6.5, 3.3, 1],
]
tree_classification = tree(data, max_depth=3, min_size=2, metric=epc_c, is_clasification=True)


In [7]:
# Varience 
data_regression = [
    [2.5, 3.5, 1.5, 10.0],
    [1.0, 2.0, 1.0, 12.0],
    [3.0, 4.0, 2.0, 15.0],
    [4.5, 5.5, 2.5, 20.0],
    [5.0, 6.0, 3.0, 25.0],
    [6.5, 7.5, 3.5, 30.0],
    [1.5, 2.5, 1.2, 11.0],
    [7.0, 8.0, 4.0, 35.0],
    [8.0, 9.0, 5.0, 40.0],
    [4.0, 2.0, 1.8, 18.0],
    [9.0, 7.5, 6.0, 45.0],
    [5.5, 6.5, 3.3, 28.0],
]

tree_regression = tree(data_regression, max_depth=3, min_size=2, metric=vari_red, is_clasification=False)


In [8]:
import pprint

print("Árbol de Clasificación:")
pprint.pprint(tree_classification)

print("\nÁrbol de Regresión:")
pprint.pprint(tree_regression)

Árbol de Clasificación:
{'index': 2,
 'left': {'index': 2,
          'left': {'index': 2, 'left': 0, 'right': 0, 'value': 1.2},
          'right': {'index': 2, 'left': 0, 'right': 0, 'value': 1.8},
          'value': 1.8},
 'right': {'index': 2, 'left': 1, 'right': 1, 'value': 3.3},
 'value': 3.3}

Árbol de Regresión:
{'index': 2,
 'left': {'index': 2,
          'left': {'index': 2, 'left': 12.0, 'right': 10.0, 'value': 1.2},
          'right': {'index': 2, 'left': 25.0, 'right': 25.0, 'value': 1.8},
          'value': 1.8},
 'right': {'index': 2, 'left': 35.0, 'right': 35.0, 'value': 3.3},
 'value': 3.3}


In [9]:
from graphviz import Digraph
def tree_plot(node, graph=None, parent=None, label=None):
    if graph is None:
        graph = Digraph(format='png')
        graph.attr(dpi='300')

    if isinstance(node, (int, float)):  
        graph.node(str(id(node)), f'Leaf: {node}', shape='box')
    else:
        condition = f'Feature {node["index"]} < {node["value"]:.2f}'
        graph.node(str(id(node)), condition)

    if parent is not None:
        graph.edge(parent, str(id(node)), label=label)

    if isinstance(node, dict): 
        if 'left' in node:
            tree_plot(node['left'], graph, str(id(node)), 'True')
        if 'right' in node:
            tree_plot(node['right'], graph, str(id(node)), 'False')

    return graph

In [12]:
# Grafica Clasificacion 
tree_classification = tree(data, max_depth=3, min_size=2, metric=epc_c, is_clasification=True)
graph_classification = tree_plot(tree_classification)
graph_classification.render("Arbol de Clasificacion", view=True)  


'Arbol de Clasificacion.png'

In [13]:
# Grafica de regresion 
tree_regression = tree(data_regression, max_depth=3, min_size=2, metric=vari_red, is_clasification=False)
graph_regression = tree_plot(tree_regression)
graph_regression.render("Arbol de Regresion", view=True)

'Arbol de Regresion.png'